In [95]:
from PIL import Image
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad, unpad
from salsa20 import XSalsa20_xor
from os import urandom


In [45]:
def string_to_binary(message):
    encodeM = str(message).encode()
    bin_int = int.from_bytes(encodeM, 'big')
    bin_string = bin(bin_int)[2:]
    return "0" + bin_string

In [3]:
def encode_message(pixels, binString):
    # loop through each pixel
    for i in range(len(pixels)):
        for j in range(len(pixels[i])):
            if (binString == ''):
                return pixels
            # get the pixel value this is in binary already
            pixel = pixels[i][j]
            # print(pixel)
            # get first two characters of binString to be implanted into pixel
            binStringC = binString[:2]
            # remove first two characters of binString to the message will be eaten while looping
            binString = binString[2:]
            # get the first 6 character of binC then add the two bits of binStringC to the end
            pixel = pixel[:6] + binStringC
            #set the new pixel value
            pixels[i][j] = pixel
    # return pixels


 
                

In [4]:
def decode_message(pixelsWithMessage):
    binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixelsWithMessage]
    binPixels = [list(pixel[-2:] for pixel in pixel) for pixel in binaryPixels]
    # binPixels
    # iterate through each binPixels and concatenate them into a string
    binString = ''.join([''.join(pixel) for pixel in binPixels])
    #split into 8 bit chunks
    binString = [binString[i:i+8] for i in range(0, len(binString), 8)]
    # print(binString)

    # iterate through each binString and convert to int
    binString = [int(b, 2) for b in binString]
    # binString
    # turn ascii numbers to ascii characters
    binString = [chr(b) for b in binString]

    binString = ''.join(binString)
    return binString

In [5]:
def binary_to_string(binString):
    binary_int = int(binString, 2)  
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = binary_array.decode()
    return ascii_text

In [6]:
def canEncode(pixels, message):
    LengthM = len(string_to_binary(message))
    LengthP = len(pixels) *3 *2
    print("message length: ", LengthM)
    print("pixels length: ", LengthP)
    if LengthM > LengthP:
        return False, LengthM, LengthP
    return True, LengthM, LengthP

# Hide info and create photo

In [17]:
def hide_Message(Photo1, hiddenMessage, photoName):
    # turn txt file to binary
    print("creating binary string")
    binMessage = string_to_binary(hiddenMessage)
    # turn photo to pixel representation
    pixels = Photo1.getdata()
    # turn pixel representation into a list of lists
    binaryPixels = [list(bin(p)[2:].rjust(8,'0') for p in pixel) for pixel in pixels]
    # implant the message
    print("encoding message")
    print(type(binMessage))
    print(type(binaryPixels))
    newPhotoPixelsBinary = encode_message(binaryPixels, binMessage)
    # turn the binary pixels back into rgb values
    print("converting back to rgb")
    newPhotoPixels = [tuple(int(p,2) for p in pixel) for pixel in newPhotoPixelsBinary]
    # turn the rgb values back into a photo
    newPhoto = Image.new("RGB", Photo1.size)
    newPhoto.putdata(newPhotoPixels)
    # save the photo
    finalFilename = ".".join([photoName, "png"])
    newPhoto.save(finalFilename)



## no encyrption

In [93]:
Photo1 = Image.open("startImages/IMG_4279.JPG")
Photo2 = Image.open('startImages/Photo2.jpg')

hiddenMessage = open('ThingsToHide/LoTR.txt', 'r').read()
hiddenMessage = str(hiddenMessage)
# testB = string_to_binary(hiddenMessage)

In [94]:
if canEncode(Photo1.getdata(), hiddenMessage):
    hide_Message(Photo1, hiddenMessage, "encodedImages/notEncypted")

message length:  8204672
pixels length:  72999936
creating binary string
encoding message
<class 'str'>
<class 'list'>
converting back to rgb


In [96]:
notEncryptedMessage = Image.open('encodedImages/notEncypted.png')
pixelsWithMessage = notEncryptedMessage.getdata()
LBSMessage = decode_message(pixelsWithMessage)
# LBSMessage
# save the message to a file
with open('decodedMessages/LSBMessage_noEncryption.txt', 'w') as f:
    f.write(LBSMessage)


# with encryption

AES

In [46]:
Photo1 = Image.open("startImages/Photo1.jpg")
hiddenMessage = open('ThingsToHide/test.txt', 'r').read()
hiddenMessage = str(hiddenMessage)

In [97]:
#this works can not decrpyt from it
data = bytes(hiddenMessage, 'utf-8')
key = b'\x14\xdaE\xee-Tn\xa3\xf6\xce<\x05,\x00\xadF'
print(" this is the key: ", key)
cipher = AES.new(key, AES.MODE_ECB)
#pad the data
padded_text = pad(data, AES.block_size)
# print(padded_text)
encrypted_message = cipher.encrypt(padded_text)

 this is the key:  b'\x14\xdaE\xee-Tn\xa3\xf6\xce<\x05,\x00\xadF'


test to make sure decrypts correctly

In [48]:
decipher = AES.new(key, AES.MODE_ECB)
decrypted_message = decipher.decrypt(encrypted_message)
unpadded_decrypted_message = unpad(decrypted_message, 16)   

# print(unpadded_decrypted_message)

In [49]:
# hide the message in the photo
if canEncode(Photo1.getdata(), str(encrypted_message)):
    hide_Message(Photo1, str(encrypted_message), "encodedImages/encrypted")

message length:  316744
pixels length:  7588800
creating binary string
encoding message
<class 'str'>
<class 'list'>
converting back to rgb


In [50]:
hiddenImgEncrypted = Image.open('encodedImages/encrypted.png')

pixelsWithMessageEncrypted = hiddenImgEncrypted.getdata()

In [51]:
encrypted_message_from_Img = decode_message(pixelsWithMessageEncrypted)#decode

In [53]:
encrypted_bytes = bytes(encrypted_message_from_Img, 'utf-8')

decipher = AES.new(key, AES.MODE_ECB)

decrypted_message = decipher.decrypt(encrypted_bytes)

dectpt_unpad= unpad(decrypted_message, 16)


ValueError: Data must be aligned to block boundary in ECB mode

Salsa20

In [54]:
Photo1 = Image.open("startImages/Photo1.jpg")
hiddenMessage = open('ThingsToHide/test.txt', 'r').read()
# hiddenMessage = str(hiddenMessage)

In [56]:
IV = urandom(24)
KEY = b'*secret**secret**secret**secret*'
ciphertext = XSalsa20_xor(bytes(hiddenMessage, 'utf-8'), IV, KEY)
# print(XSalsa20_xor(ciphertext, IV, KEY).decode())

In [57]:
# str(ciphertext)
string_to_binary(ciphertext[:1])

'01100010001001110011111000100111'

In [58]:
if canEncode(Photo1.getdata(), str(ciphertext)):
    hide_Message(Photo1, str(ciphertext), "encodedImages/encrypted")

message length:  317056
pixels length:  7588800
creating binary string
encoding message
<class 'str'>
<class 'list'>
converting back to rgb


In [25]:
hiddenImgEncrypted = Image.open('encodedImages/encrypted.png')
pixelsWithMessageEncrypted = hiddenImgEncrypted.getdata()

In [26]:
encrypted_message_from_Img = decode_message(pixelsWithMessageEncrypted)#decode
# type(encrypted_message_from_Img)

str

In [59]:
# encrypted_message_from_Img

In [62]:
encrypted_bytes = bytes(encrypted_message_from_Img, 'utf-8')
decrypted_msg = XSalsa20_xor(encrypted_bytes, IV, KEY).decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 3: invalid start byte